In [1]:
import os
import numpy as np
import pandas as pd
import torch
import os
import json
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm
import glob
import pickle
DATA_DIR = '/mnt/chicm/data/open-images/relation'
DETECT_DATA_DIR = '/mnt/chicm/data/open-images/detect'

In [2]:
!ls -lh

total 21G
-rw-rw-r-- 1 core core  3.2G Aug 13 11:40 all_models_team_submission_v1_large_20190813.csv
-rw-rw-r-- 1 core core  850M Aug 22 14:21 all_models_team_submission_v1_large_20190813.csv.zip
drwxrwxr-x 4 core core  4.0K Aug 10 15:30 catboost_info
-rw-rw-r-- 1 core core   852 Aug  4 15:31 cat_feature_index.c1a4dabd-715a703f-75517141-d81266d1.tmp
-rw-rw-r-- 1 core core   852 Aug  4 15:04 cat_feature_index.e36fdba6-6cd399f6-229ea45f-ea07b906.tmp
-rw-rw-r-- 1 core core  2.2M Aug 20 15:31 detect-57-ensemble.ipynb
-rw-rw-r-- 1 core core  2.2M Aug 22 16:05 detect-57-ensemble-level2.ipynb
-rw-rw-r-- 1 core core  1.3M Aug 21 07:32 eda.ipynb
-rw-rw-r-- 1 core core  421M Aug  8 09:52 ensemble_detect_57_0808_1.csv
-rw-rw-r-- 1 core core  268M Aug  8 09:56 ensemble_detect_57_0808_1_top50.csv
-rw-rw-r-- 1 core core  421M Aug  8 15:04 ensemble_detect_57_0808_2_with_weights.csv
-rw-rw-r-- 1 core core  399M Aug 19 17:09 ensemble_detect_57_0819_1.csv
-rw-rw-r-- 1 core core  399M Aug 19 17:10 ensemb

In [3]:
df_box = pd.read_csv(os.path.join(DATA_DIR, 'challenge-2019-train-vrd-bbox.csv'))
df_box.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax,IsGroupOf
0,780d8a470d144fa0,/m/04yx4,0.000000,0.527143,0.029979,0.997859,0
1,780d8a470d144fa0,/m/04yx4,0.291429,0.687143,0.128480,0.997859,0
2,780d8a470d144fa0,/m/04yx4,0.590000,0.965714,0.164882,0.997859,0
3,780db0a7d07a4a20,/m/026t6,0.000000,1.000000,0.345625,0.990000,-1
4,780db0a7d07a4a20,/m/026t6,0.271667,0.626667,0.803125,0.999375,0


In [4]:
print(df_box.shape, len(df_box.ImageID.unique()), len(df_box.LabelName.unique()))

(3290070, 7) 790478 57


In [5]:
df_vrd = pd.read_csv(os.path.join(DATA_DIR, 'challenge-2019-train-vrd.csv'))
df_vrd.head()

,ImageID,LabelName1,LabelName2,XMin1,XMax1,YMin1,YMax1,XMin2,XMax2,YMin2,YMax2,RelationshipLabel
0,fe58ec1b06db2bb7,/m/04bcr3,/m/083vt,0.00500,0.033125,0.580000,0.627778,0.00500,0.033125,0.580000,0.627778,is
1,82d16a22f703df5c,/m/04dr76w,/m/02gy9n,0.61200,0.735000,0.418919,0.950450,0.61200,0.735000,0.418919,0.950450,is
2,b54d41beabcfd900,/m/01mzpv,/m/083vt,0.37250,0.399375,0.706413,0.778557,0.37250,0.399375,0.706413,0.778557,is
3,4b6a08cc110d7275,/m/01mzpv,/m/01y9k5,0.17125,0.255625,0.557500,0.749167,0.20750,0.683125,0.611667,0.999167,at
4,0144cfbb726f4c72,/m/01mzpv,/m/04bcr3,0.85500,0.950000,0.561667,0.609167,0.82875,0.999375,0.568333,0.673333,at


In [6]:
print(df_vrd.shape, len(df_vrd.ImageID.unique()), len(df_vrd.LabelName1.unique()),\
      len(df_vrd.LabelName2.unique()), len(df_vrd.RelationshipLabel.unique()))

(374768, 12) 100522 32 58 10


In [7]:
df_vrd.loc[df_vrd.ImageID=='fe58ec1b06db2bb7']

,ImageID,LabelName1,LabelName2,XMin1,XMax1,YMin1,YMax1,XMin2,XMax2,YMin2,YMax2,RelationshipLabel
0,fe58ec1b06db2bb7,/m/04bcr3,/m/083vt,0.005000,0.033125,0.580000,0.627778,0.005000,0.033125,0.580000,0.627778,is
77712,fe58ec1b06db2bb7,/m/04bcr3,/m/083vt,0.071875,0.121875,0.597778,0.680000,0.071875,0.121875,0.597778,0.680000,is
171028,fe58ec1b06db2bb7,/m/04bcr3,/m/083vt,0.124375,0.217500,0.612222,0.712222,0.124375,0.217500,0.612222,0.712222,is
230777,fe58ec1b06db2bb7,/m/04bcr3,/m/083vt,0.185000,0.308125,0.593333,0.768889,0.185000,0.308125,0.593333,0.768889,is
284575,fe58ec1b06db2bb7,/m/04bcr3,/m/083vt,0.513750,0.903125,0.680000,0.998889,0.513750,0.903125,0.680000,0.998889,is
362652,fe58ec1b06db2bb7,/m/04bcr3,/m/083vt,0.308750,0.517500,0.632222,0.864444,0.308750,0.517500,0.632222,0.864444,is
370350,fe58ec1b06db2bb7,/m/04bcr3,/m/083vt,0.038125,0.071875,0.582222,0.662222,0.038125,0.071875,0.582222,0.662222,is


In [8]:
df_vrd['labels'] = df_vrd.LabelName1.str.cat(df_vrd.LabelName2, sep=' ')
df_vrd.head()

,ImageID,LabelName1,LabelName2,XMin1,XMax1,YMin1,YMax1,XMin2,XMax2,YMin2,YMax2,RelationshipLabel,labels
0,fe58ec1b06db2bb7,/m/04bcr3,/m/083vt,0.00500,0.033125,0.580000,0.627778,0.00500,0.033125,0.580000,0.627778,is,/m/04bcr3 /m/083vt
1,82d16a22f703df5c,/m/04dr76w,/m/02gy9n,0.61200,0.735000,0.418919,0.950450,0.61200,0.735000,0.418919,0.950450,is,/m/04dr76w /m/02gy9n
2,b54d41beabcfd900,/m/01mzpv,/m/083vt,0.37250,0.399375,0.706413,0.778557,0.37250,0.399375,0.706413,0.778557,is,/m/01mzpv /m/083vt
3,4b6a08cc110d7275,/m/01mzpv,/m/01y9k5,0.17125,0.255625,0.557500,0.749167,0.20750,0.683125,0.611667,0.999167,at,/m/01mzpv /m/01y9k5
4,0144cfbb726f4c72,/m/01mzpv,/m/04bcr3,0.85500,0.950000,0.561667,0.609167,0.82875,0.999375,0.568333,0.673333,at,/m/01mzpv /m/04bcr3


In [9]:
len(df_vrd.loc[df_vrd.RelationshipLabel=='is'].labels.unique())

42

In [10]:
counts = df_vrd.loc[df_vrd.RelationshipLabel=='is'].labels.value_counts()

In [11]:
type(counts)

pandas.core.series.Series

In [16]:
dir(counts)

['T',
 '_AXIS_ALIASES',
 '_AXIS_IALIASES',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_SLICEMAP',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_prepare__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__bytes__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 

In [26]:
is_dict = {}
for rel in counts.index:
    k, v = rel.split(' ')
    #print(k,':', v)
    #break
    if k not in is_dict:
        is_dict[k] = []
    is_dict[k].append(v)

In [27]:
is_dict

{'/m/04bcr3': ['/m/083vt', '/m/05z87', '/m/02gy9n'],
 '/m/01mzpv': ['/m/083vt', '/m/05z87', '/m/04lbp'],
 '/m/04dr76w': ['/m/02gy9n', '/m/05z87'],
 '/m/0342h': ['/m/083vt'],
 '/m/01y9k5': ['/m/083vt'],
 '/m/0cvnqh': ['/m/083vt', '/m/05z87'],
 '/m/078n6m': ['/m/083vt', '/m/02gy9n', '/m/05z87'],
 '/m/026t6': ['/m/083vt'],
 '/m/07y_7': ['/m/083vt'],
 '/m/080hkjn': ['/m/0dnr7', '/m/04lbp'],
 '/m/03m3pdh': ['/m/0dnr7', '/m/04lbp'],
 '/m/03ssj5': ['/m/083vt'],
 '/m/01940j': ['/m/0dnr7', '/m/04lbp'],
 '/m/05r5c': ['/m/083vt', '/m/05z87'],
 '/m/01s55n': ['/m/0dnr7', '/m/04lbp', '/m/05z87'],
 '/m/01_5g': ['/m/083vt', '/m/05z87'],
 '/m/071p9': ['/m/083vt'],
 '/m/0584n8': ['/m/0dnr7', '/m/04lbp'],
 '/m/0cmx8': ['/m/05z87'],
 '/m/02p5f1q': ['/m/05z87', '/m/02gy9n'],
 '/m/02jvh9': ['/m/05z87', '/m/02gy9n', '/m/083vt'],
 '/m/0dt3t': ['/m/05z87'],
 '/m/04ctx': ['/m/05z87']}

In [36]:
len(is_dict)

23

In [28]:
df_box.loc[df_box.ImageID=='fe58ec1b06db2bb7']

,ImageID,LabelName,XMin,XMax,YMin,YMax,IsGroupOf
2055608,fe58ec1b06db2bb7,/m/04bcr3,0.185000,0.308125,0.593333,0.768889,0
2055609,fe58ec1b06db2bb7,/m/04bcr3,0.005000,0.033125,0.580000,0.627778,0
2055610,fe58ec1b06db2bb7,/m/04bcr3,0.038125,0.071875,0.582222,0.662222,0
2055611,fe58ec1b06db2bb7,/m/04bcr3,0.071875,0.121875,0.597778,0.680000,0
2055612,fe58ec1b06db2bb7,/m/04bcr3,0.124375,0.217500,0.612222,0.712222,0
2055613,fe58ec1b06db2bb7,/m/04bcr3,0.308750,0.517500,0.632222,0.864444,0
2055614,fe58ec1b06db2bb7,/m/04bcr3,0.513750,0.903125,0.680000,0.998889,0


In [22]:
df_vrd.RelationshipLabel.value_counts()

is                194142
at                111493
on                 31604
holds              20986
plays               8932
interacts_with      3756
inside_of           2392
wears                836
hits                 593
under                 34
Name: RelationshipLabel, dtype: int64

In [25]:
df_vrd.loc[df_vrd.RelationshipLabel=='is'].LabelName2.value_counts()

/m/083vt     117165
/m/05z87      36140
/m/02gy9n     20644
/m/04lbp      14411
/m/0dnr7       5782
Name: LabelName2, dtype: int64

In [43]:
df_vrd.loc[df_vrd.RelationshipLabel=='is'].LabelName1.value_counts()

/m/01mzpv     70456
/m/04bcr3     48975
/m/04dr76w    27270
/m/0342h      13444
/m/01y9k5      7472
/m/0cvnqh      5089
/m/080hkjn     3820
/m/078n6m      3696
/m/026t6       2906
/m/07y_7       2550
/m/03m3pdh     2129
/m/03ssj5      1370
/m/01940j      1276
/m/05r5c       1182
/m/01s55n       867
/m/01_5g        574
/m/0584n8       290
/m/071p9        204
/m/02p5f1q      195
/m/0cmx8        151
/m/02jvh9       141
/m/0dt3t         55
/m/04ctx         30
Name: LabelName1, dtype: int64

In [9]:
classes_is = df_vrd.loc[df_vrd.RelationshipLabel=='is'].LabelName1.unique().tolist()

In [101]:
np.array(classes_is)

array(['/m/04bcr3', '/m/04dr76w', '/m/01mzpv', '/m/078n6m', '/m/0342h',
       '/m/03m3pdh', '/m/03ssj5', '/m/01_5g', '/m/01y9k5', '/m/0cvnqh',
       '/m/07y_7', '/m/071p9', '/m/05r5c', '/m/01940j', '/m/01s55n',
       '/m/080hkjn', '/m/026t6', '/m/02p5f1q', '/m/0cmx8', '/m/0dt3t',
       '/m/0584n8', '/m/04ctx', '/m/02jvh9'], dtype='<U10')

In [11]:
df_classes = pd.read_csv(os.path.join(DATA_DIR, 'challenge-2019-classes-vrd.csv'), names=['classes', 'desc'])
df_classes.head()

,classes,desc
0,/m/05r5c,Piano
1,/m/01599,Beer
2,/m/01_5g,Chopsticks
3,/m/01yrx,Cat
4,/m/04_sv,Motorcycle


In [16]:
df_detect_classes = pd.read_csv(os.path.join(DETECT_DATA_DIR, 'challenge-2019-classes-description-500.csv'), names=['classes', 'desc2'])
df_detect_classes.head()

,classes,desc2
0,/m/061hd_,Infant bed
1,/m/06m11,Rose
2,/m/03120,Flag
3,/m/01kb5b,Flashlight
4,/m/0120dh,Sea turtle


In [17]:
df_join = df_classes.set_index('classes').join(df_detect_classes.set_index('classes'), on='classes')
df_join.head(50)

,desc,desc2
classes,,
/m/05r5c,Piano,Piano
/m/01599,Beer,Beer
/m/01_5g,Chopsticks,Chopsticks
/m/01yrx,Cat,Cat
/m/04_sv,Motorcycle,Motorcycle
/m/0h8my_4,Tennis racket,Tennis racket
/m/01226z,Football,Football
/m/050k8,Mobile phone,Mobile phone
/m/0l14j_,Flute,Flute


In [18]:
df_join.shape

(57, 2)

In [29]:
!ls -lh

total 21G
-rw-rw-r-- 1 core core  3.2G Aug 13 11:40 all_models_team_submission_v1_large_20190813.csv
-rw-rw-r-- 1 core core  850M Aug 22 14:21 all_models_team_submission_v1_large_20190813.csv.zip
drwxrwxr-x 4 core core  4.0K Aug 10 15:30 catboost_info
-rw-rw-r-- 1 core core   852 Aug  4 15:31 cat_feature_index.c1a4dabd-715a703f-75517141-d81266d1.tmp
-rw-rw-r-- 1 core core   852 Aug  4 15:04 cat_feature_index.e36fdba6-6cd399f6-229ea45f-ea07b906.tmp
-rw-rw-r-- 1 core core  2.2M Aug 20 15:31 detect-57-ensemble.ipynb
-rw-rw-r-- 1 core core  2.2M Aug 22 16:05 detect-57-ensemble-level2.ipynb
-rw-rw-r-- 1 core core  1.3M Aug 21 07:32 eda.ipynb
-rw-rw-r-- 1 core core  421M Aug  8 09:52 ensemble_detect_57_0808_1.csv
-rw-rw-r-- 1 core core  268M Aug  8 09:56 ensemble_detect_57_0808_1_top50.csv
-rw-rw-r-- 1 core core  421M Aug  8 15:04 ensemble_detect_57_0808_2_with_weights.csv
-rw-rw-r-- 1 core core  399M Aug 19 17:09 ensemble_detect_57_0819_1.csv
-rw-rw-r-- 1 core core  399M Aug 19 17:10 ensemb

In [30]:
df_det = pd.read_csv('ensemble_detect_57_0820_1_top100.csv')
df_det.head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,/m/04yx4 0.8209846 0.3756532 0.2369880 0.75013...
1,763c72626bf24534,/m/04yx4 0.9761059 0.0059799 0.1897154 0.24897...
2,7540c162180973b1,/m/04yx4 0.6516410 0.7939431 0.1332737 0.85725...
3,c4e522dce52cb02f,/m/04yx4 0.9427195 0.3922202 0.2638703 0.54933...
4,cafcd9496c808085,/m/0k4j 0.0011794 0.7955955 0.0142576 0.999023...


In [33]:
def get_det(pred_str):
    dets = []
    det = []
    for i, e in enumerate(pred_str.split(' ')):
        if i % 6 == 0:
            det = []
        det.append(e)
        if (i+1) % 6 == 0:
            if det[0] in set(is_dict): #and float(det[1]) > 0.1:
                dets.append(det)
            
    return dets

In [34]:
df_det['dets'] = df_det.PredictionString.map(lambda x: get_det(str(x)))
df_det.head()

,ImageId,PredictionString,dets
0,21135d5ed0e7a0b7,/m/04yx4 0.8209846 0.3756532 0.2369880 0.75013...,"[[/m/01mzpv, 0.2925354, 0.0173653, 0.6797319, ..."
1,763c72626bf24534,/m/04yx4 0.9761059 0.0059799 0.1897154 0.24897...,"[[/m/01mzpv, 0.0251931, 0.0004263, 0.3061501, ..."
2,7540c162180973b1,/m/04yx4 0.6516410 0.7939431 0.1332737 0.85725...,"[[/m/026t6, 0.0399812, 0.9395664, 0.4553831, 0..."
3,c4e522dce52cb02f,/m/04yx4 0.9427195 0.3922202 0.2638703 0.54933...,"[[/m/01mzpv, 0.0151644, 0.8399238, 0.6316088, ..."
4,cafcd9496c808085,/m/0k4j 0.0011794 0.7955955 0.0142576 0.999023...,"[[/m/04bcr3, 0.1576493, 0.0143700, 0.0000000, ..."


In [117]:
len(df_det.iloc[102].dets)

15

In [83]:
df_vrd.loc[df_vrd.RelationshipLabel=='is'].LabelName2.unique()

array(['/m/083vt', '/m/02gy9n', '/m/05z87', '/m/04lbp', '/m/0dnr7'],
      dtype=object)

In [37]:
def get_rel_pred(dets):
    preds = []
    for det in dets:
        #for label2 in ['/m/083vt', '/m/02gy9n', '/m/05z87', '/m/04lbp', '/m/0dnr7']:
        label1 = det[0]
        #for label2 in ['/m/083vt']:
        for label2 in (is_dict[label1])[:2]:
            preds.append(det[1]) # conf
            preds.append(det[0]) # label1
            preds.append(det[2])
            preds.append(det[3])
            preds.append(det[4])
            preds.append(det[5])

            preds.append(label2) #label2
            preds.append(det[2])
            preds.append(det[3])
            preds.append(det[4])
            preds.append(det[5])
            preds.append('is')
    preds = [str(x) for x in preds]
    return ' '.join(preds)

In [38]:
df_det.PredictionString = df_det.dets.map(lambda x: get_rel_pred(x))

In [39]:
df_det.head()

,ImageId,PredictionString,dets
0,21135d5ed0e7a0b7,0.2925354 /m/01mzpv 0.0173653 0.6797319 0.4048...,"[[/m/01mzpv, 0.2925354, 0.0173653, 0.6797319, ..."
1,763c72626bf24534,0.0251931 /m/01mzpv 0.0004263 0.3061501 0.0632...,"[[/m/01mzpv, 0.0251931, 0.0004263, 0.3061501, ..."
2,7540c162180973b1,0.0399812 /m/026t6 0.9395664 0.4553831 0.99871...,"[[/m/026t6, 0.0399812, 0.9395664, 0.4553831, 0..."
3,c4e522dce52cb02f,0.0151644 /m/01mzpv 0.8399238 0.6316088 0.9945...,"[[/m/01mzpv, 0.0151644, 0.8399238, 0.6316088, ..."
4,cafcd9496c808085,0.1576493 /m/04bcr3 0.0143700 0.0000000 0.9863...,"[[/m/04bcr3, 0.1576493, 0.0143700, 0.0000000, ..."


In [40]:
df_det.to_csv('sub_is_top2_0823.csv', columns=['ImageId', 'PredictionString'], index=False)

In [41]:
!kaggle competitions submit -c open-images-2019-visual-relationship -f sub_is_top2_0823.csv -m "submit"

100%|████████████████████████████████████████| 634M/634M [00:22<00:00, 29.6MB/s]
Successfully submitted to Open Images 2019 - Visual Relationship

In [123]:
!ls -lh

total 1.8G
-rw-rw-r-- 1 core core  1.3M Jul 25 11:20 eda.ipynb
-rw-rw-r-- 1 core core  372K Jul 25 12:35 Is_classifier_42.ipynb
-rw-rw-r-- 1 core core  370K Jul 25 05:24 Is_classifier.ipynb
-rw-rw-r-- 1 core core  266M Jul 22 16:57 sub_detect_0722_1.csv
-rw-rw-r-- 1 core core  252M Jul 24 17:01 sub_detect_0724.csv
-rw-rw-r-- 1 core core  157M Jul 25 17:15 sub_is_083vt_0724.csv
-rw-rw-r-- 1 core core  777M Jul 24 17:06 sub_is_all_label2_0724.csv
-rw-rw-r-- 1 core core  138M Jul 22 17:06 sub_is_th0.01_0723.csv
-rw-rw-r-- 1 core core   33M Jul 22 17:08 sub_is_th0.1_0723.csv
-rw-rw-r-- 1 core core  157M Jul 22 17:20 sub_is_th0.1_all_label2_0723.csv
-rw-rw-r-- 1 core core 1011K Jul 22 16:59 submit-detect.ipynb
-rw-r--r-- 1 core core   81K Jul 25 17:14 submit.ipynb


In [93]:
!head -n 2 sub_is_all_label2_0724.csv

ImageId,PredictionString
21135d5ed0e7a0b7,0.3736828 /m/01mzpv 0.0550915 0.7031765 0.3570746 0.9980469 /m/083vt 0.0550915 0.7031765 0.3570746 0.9980469 is 0.3736828 /m/01mzpv 0.0550915 0.7031765 0.3570746 0.9980469 /m/02gy9n 0.0550915 0.7031765 0.3570746 0.9980469 is 0.3736828 /m/01mzpv 0.0550915 0.7031765 0.3570746 0.9980469 /m/05z87 0.0550915 0.7031765 0.3570746 0.9980469 is 0.3736828 /m/01mzpv 0.0550915 0.7031765 0.3570746 0.9980469 /m/04lbp 0.0550915 0.7031765 0.3570746 0.9980469 is 0.3736828 /m/01mzpv 0.0550915 0.7031765 0.3570746 0.9980469 /m/0dnr7 0.0550915 0.7031765 0.3570746 0.9980469 is 0.1037584 /m/01mzpv 0.6970934 0.4817260 0.9553819 0.9970111 /m/083vt 0.6970934 0.4817260 0.9553819 0.9970111 is 0.1037584 /m/01mzpv 0.6970934 0.4817260 0.9553819 0.9970111 /m/02gy9n 0.6970934 0.4817260 0.9553819 0.9970111 is 0.1037584 /m/01mzpv 0.6970934 0.4817260 0.9553819 0.9970111 /m/05z87 0.6970934 0.4817260 0.9553819 0.9970111 is 0.1037584 /m/01mzpv 0.6970934 0.4817260 0.9553819 0.9970111 

In [102]:
## test
img_ids, labels, confs, xmins, ymins, xmaxs, ymaxs = [], [], [], [], [], [], []

for img_id, dets in tqdm(zip(df_det.ImageId.values, df_det.dets.values)):
    #print(img_id)
    #print(dets)
    for det in dets:
        #det_list.append((img_id, det))
        img_ids.append(img_id)
        labels.append(det[0])
        confs.append(det[1])
        xmins.append(det[2])
        ymins.append(det[3])
        xmaxs.append(det[4])
        ymaxs.append(det[5])

99999it [00:01, 64080.67it/s]


In [103]:
len(img_ids)

1449428

In [106]:
df_pred = pd.DataFrame({
    'ImageID': img_ids,
    'LabelName': labels,
    'Confidence': confs,
    'XMin': xmins,
    'YMin': ymins,
    'XMax': xmaxs,
    'YMax': ymaxs
})

In [111]:
df_pred.head()

,ImageID,LabelName,Confidence,XMin,YMin,XMax,YMax
0,21135d5ed0e7a0b7,/m/01mzpv,0.3736828,0.0550915,0.7031765,0.3570746,0.9980469
1,21135d5ed0e7a0b7,/m/01mzpv,0.1037584,0.6970934,0.4817260,0.9553819,0.9970111
2,21135d5ed0e7a0b7,/m/01mzpv,0.0333073,0.7835019,0.2043150,0.9863467,0.9684071
3,21135d5ed0e7a0b7,/m/01mzpv,0.0309122,0.6829767,0.4272454,0.8355507,0.9967163
4,21135d5ed0e7a0b7,/m/04bcr3,0.3338915,0.3670477,0.7229147,0.4771416,0.8875893


In [112]:
for index, row in df_pred.iterrows():
    print(row)
    #print(row.ImageID)
    break

ImageID       21135d5ed0e7a0b7
LabelName            /m/01mzpv
Confidence           0.3736828
XMin                 0.0550915
YMin                 0.7031765
XMax                 0.3570746
YMax                 0.9980469
Name: 0, dtype: object
